Play with getting a faster way to get ground coordinates

In [48]:
from ecostress import *
from geocal import *
%load_ext Cython

In [2]:
igc = read_shelve("../python/igc.xml")

In [3]:
igc.orbit

<geocal_swig.hdf_orbit.HdfOrbit_Eci_TimeJ2000; proxy of <Swig Object of type 'boost::shared_ptr< GeoCal::HdfOrbit< GeoCal::Eci,GeoCal::TimeJ2000Creator > > *' at 0x7f5a7a5f9450> >

In [9]:
print(igc.time_table.time(ImageCoordinate(0,0))[0])
print(igc.time_table.time(ImageCoordinate(255,0))[0])

2015-01-24T20:42:51.230216Z
2015-01-24T20:42:51.230216Z


In [71]:
class GroundCoorArr(object):
    def __init__(self,igc):
        self.time_table = igc.time_table
        self.igc = igc
        self.slv = [igc.camera.sc_look_vector(FrameCoordinate(i, 0), igc.band) 
                   for i in range(igc.camera.number_line(igc.band))]
        
    def ground_coor_arr(self, start_line):
        self.res = np.empty((len(self.slv), self.igc.number_sample,3))
        self.dist = np.empty((len(self.slv)))
        ms = int(self.igc.number_sample / 2)
        self.ground_coor_arr_samp(start_line, ms, 
                                  initial_samp = True)
        self.dist_middle = self.dist.copy()
        for smp in range(ms + 1, self.igc.number_sample):
            print("Doing ", smp)
            self.ground_coor_arr_samp(start_line, smp)
        self.dist[:] = self.dist_middle
        for smp in range(ms - 1, ms - 10, -1):
            print("Doing ", smp)
            self.ground_coor_arr_samp(start_line, smp)
        return self.res
        
    def ground_coor_arr_samp(self, start_line, sample, initial_samp = False):
        t, fc = self.time_table.time(ImageCoordinate(start_line, sample))
        od = self.igc.orbit_data(t, sample)
        cf = od.position_cf
        for i, sl in enumerate(self.slv):
            lv = od.cf_look_vector(sl)
            if(initial_samp):
                self.res[i,sample,:] = self.igc.dem.intersect(cf, lv, self.igc.resolution, 
                                                         self.igc.max_height).position
            else:
                start_dist = self.dist[i]
                if(i - 1 >= 0):
                    start_dist = min(start_dist, self.dist[i-1])
                if(i + 1 < self.dist.shape[0]):
                    start_dist = min(start_dist, self.dist[i-1])
                self.res[i,sample,:] = self.igc.dem.intersect_start_length(cf, lv, 
                                            self.igc.resolution,  start_dist).position
            self.dist[i] = np.linalg.norm(self.res[i,sample,:] - cf.position)
            



In [65]:
c = GroundCoorArr(igc)
t = c.ground_coor_arr(0)
print(t[0,2700,:])

AttributeError: '_cython_magic_0995d6f2a5dc137674d73831e90935b4.GroundCoorArr' object has no attribute 'time_table'

In [45]:
distance(Ecr(*t[10,5300,:]), igc.ground_coordinate(ImageCoordinate(10, 5300)))

0.006387512412494821

In [29]:
tfull = np.zeros((256,5400, 3))

In [31]:
tfull[0,2700,:] = t[0].position